In [1]:
import pymysql
import pandas as pd
import boto3

In [2]:
host = "hyundb.cqgrmpcsglic.ap-northeast-2.rds.amazonaws.com"
port = 3306
username = 'hyunjilee'
database = 'production'
pw = ''

conn = pymysql.connect(host=host, user=username, passwd=pw,
                      db=database, port=port,
                      use_unicode=True, charset='utf8')
cursor = conn.cursor()

In [3]:
cursor.execute('SELECT * FROM artists')
a_data = cursor.fetchall()
col = [d[0] for d in cursor.description]
#cursor.execute('SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = "artists"')
#col = cursor.fetchall()
#col = [i[0] for i in col]

In [14]:
data = pd.DataFrame(a_data, columns=col)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   artist_id   502 non-null    object
 1   name        502 non-null    object
 2   followers   502 non-null    int64 
 3   popularity  502 non-null    int64 
 4   url         502 non-null    object
 5   image_url   502 non-null    object
dtypes: int64(2), object(4)
memory usage: 23.7+ KB


In [17]:
data.to_parquet('artists.parquet', engine='pyarrow',
                compression='snappy')
#dt = datetime.utcnow().strftime('%Y-%m-%d')
dt = '2021-11-01'
s3 = boto3.resource('s3', region_name='ap-northeast-2')
ob = s3.Object('hyun-artist', 'artists/dt={}/artists.parquet'.format(dt))
data = open('artists.parquet', 'rb')
ob.put(Body=data)

{'ResponseMetadata': {'RequestId': '4HK8QFRQK4GC5Z9A',
  'HostId': '02kd5Tp7eppUMDS2MaCqsqdW/7D68Z3IIJB6D5G76zOULXyomuctkDNKddf2bsRlXj5H7kfudw0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '02kd5Tp7eppUMDS2MaCqsqdW/7D68Z3IIJB6D5G76zOULXyomuctkDNKddf2bsRlXj5H7kfudw0=',
   'x-amz-request-id': '4HK8QFRQK4GC5Z9A',
   'date': 'Sun, 14 Nov 2021 22:58:36 GMT',
   'etag': '"278133b9a186181c9697b28a1a767631"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"278133b9a186181c9697b28a1a767631"'}